<a href="https://colab.research.google.com/github/FreeOnel/Deep-RL-Pacman/blob/main/PINBALL_Duelling_Double_PER_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback
import random
import gym
import cv2
from itertools import count
import numpy as np
import heapq
from collections import deque
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Conv2D, Subtract, Add, Input, Lambda
from keras.optimizers import Adam
from tqdm import tqdm
from numba import cuda

     |████████████████████████████████| 2.1MB 6.9MB/s 
     |████████████████████████████████| 163kB 23.8MB/s 
     |████████████████████████████████| 133kB 25.8MB/s 
     |████████████████████████████████| 102kB 8.1MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=0ae06905d1bb8532fc9b16c6ce695637f40ed00281067a1f9251c27af7b89a00
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=cbbffee93553c121e07dc61eb5afb0f9e16f65ece49decb8b6cb0803de17d95b
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [ ]:

class Agent:
  
    def __init__(self, state_size, action_size, n_step=3):
        self.state_size = state_size
        self.action_size = action_size
        self.buffer = []
        self.n_step = n_step
        self.n_step_buffer = deque(maxlen=n_step)
        self.count = count()
        
        self.alpha = 0.6 # Prioritization parameter
        self.gamma = 0.99 # Discount factor
        self.epsilon = 1.0 # Max Prob for Explore
        self.epsilon_min = 0.1 # Min Prob for Explore
        self.epsilon_decay = 0.995 # Decay Rate for Epsilon
        self.update_rate = 1000  # Freq of Network Update
        
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.model.summary()

    def _build_model(self):
      
        inputs = Input(shape=(self.state_size))
        
        x = Conv2D(32, (8, 8), strides=4, padding='same', activation='relu')(inputs)
        x = Conv2D(64, (4, 4), strides=2, padding='same', activation='relu')(x)
        x = Conv2D(64, (3, 3), strides=1, padding='same', activation='relu')(x)
        x = Flatten()(x)
        
        # Dueling Network
        val = Dense(1, activation='linear')(x)
        advantage = Dense(self.action_size, activation='linear')(x)
        
        # Using Mean for Advantage
        mean = Lambda(lambda x: K.mean(x, axis=1, keepdims=True))(advantage)
        advantage = Subtract()([advantage, mean])
        outputs = Add()([val, advantage])
        
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(loss='mse', optimizer=Adam())
        
        return model

    def store(self, state, action, reward, next_state, Terminal, td_error):
      
        # n-step queue for calculating return of n previous steps
        self.n_step_buffer.append((state, action, reward, next_state, Terminal))
        
        if len(self.n_step_buffer) < self.n_step:
          return
        
        l_reward, l_next_state, l_Terminal = self.n_step_buffer[-1][-3:]

        for transition in reversed(list(self.n_step_buffer)[:-1]):
            r, n_s, T = transition[-3:]

            l_reward = r + self.gamma * l_reward * (1 - T)
            l_next_state, l_Terminal = (n_s, T) if T else (l_next_state, l_Terminal)
        
        l_state, l_action = self.n_step_buffer[0][:2]

        t = (l_state, l_action, l_reward, l_next_state, l_Terminal)
        heapq.heappush(self.buffer, (-td_error, next(self.count), t))
        if len(self.buffer) > 2000:
            self.buffer = self.buffer[:-1]
        heapq.heapify(self.buffer)

    def act(self, state):
      
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        #act_values = self.model.predict(state)
        act_values = self.model(state, training=False)
        
        return np.argmax(act_values[0])
    
    # Prioritises greedily with 0.6 probability and otherwise prioritised randomly
    def replay(self, batch_size):
        
        # Semi Stochastic Prioritization
        prioritization = int(batch_size*self.alpha)
        batch_prioritized = heapq.nsmallest(prioritization, self.buffer)
        batch_uniform = random.sample(self.buffer, batch_size-prioritization)
        batch = batch_prioritized + batch_uniform
        
        batch = [e for (_, _, e) in batch]
        states = []
        targets = []
        
        for state, action, reward, next_state, Terminal in batch:
            
            if not Terminal:
                n_s = np.expand_dims(next_state.reshape(88, 80, 1), axis=0)
                # Double DQN
                m_a = np.argmax(self.model(n_s, training=False)[0])
                target = (reward + self.gamma * self.target_model(n_s, training=False)[0][m_a])
            else:
                target = reward
                
            c_s = np.expand_dims(state.reshape(88, 80, 1), axis=0)
            #target_f = self.model(c_s, training=False)
            target_f = self.model(c_s, training=False)
            proto_tensor = tf.make_tensor_proto(target_f)
            target_f = tf.make_ndarray(proto_tensor)
            target_f[0][int(action)] = target
            states.append(state)
            targets.append(target_f.reshape(self.action_size))
            
        self.model.fit(np.array(states), np.array(targets), batch_size=batch_size, epochs=1, verbose=0)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def calculate_td_error(self, state, action, reward, next_state, Terminal):
        if not Terminal:
            n_s = np.expand_dims(next_state.reshape(88, 80, 1), axis=0)
            m_a = np.argmax(self.model(n_s, training=False)[0])
            target = (reward + self.gamma * self.target_model(n_s, training=False)[0][m_a])
        else:
            target = reward

        c_s = np.expand_dims(state.reshape(88, 80, 1), axis=0)
        target_f = self.model(c_s, training=False)[0][action]
        
        return target_f - target

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def load(self, name):
        self.model.load_weights(name)
        self.target_model.set_weights(self.model.get_weights())

    def save(self, name):
        self.model.save_weights(name)

In [ ]:

# Record training with wand api
wb = True
if wb:
  wandb.init(project='PERDDQN', entity='pacman_dqn')

def preprocess(frame):
    # Got some ideas from https://github.com/ageron/tiny-dqn
    mspacman_color = np.array([210, 164, 74]).mean()
    
    # Dilate image twice with a 3x3 kernel
    kernel = np.ones((3,3),np.uint8)
    dilation = cv2.dilate(frame,kernel,iterations = 2)
    img = frame[1:176:2,::2]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img[img==mspacman_color] = 0 
    img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    return img.reshape(88, 80, 1)


env = gym.make('MsPacman-v4') # Skip 4 Frames
state_size = (88, 80, 1)
action_size = 5
agent = Agent(state_size, action_size, 5) # 5-step return

episodes = 20000
batch_size = 32
total_time = 0 
all_rewards = 0
Terminal = False

# Initializing Buffer
while len(agent.buffer) < 2000:
  state = preprocess(env.reset())
  frame_stack = deque(maxlen=4) # Deque for getting mean of 4 frames instead of stacking
  frame_stack.append(state)

  for skip in range(90): # Skip first 3 seconds of the game
      env.step(0)

  for time in range(2500):

      state = sum(frame_stack)/len(frame_stack)
  
      action = agent.act(np.expand_dims(state.reshape(88, 80, 1), axis=0))
      next_state, reward, Terminal, _ = env.step(action)

      next_state = preprocess(next_state)
      frame_stack.append(next_state)
      next_state = sum(frame_stack)/len(frame_stack)
      
      td_error = agent.calculate_td_error(state, action, reward, next_state, Terminal)
      
      agent.store(state, action, reward, next_state, Terminal, td_error)

      state = next_state

      if Terminal:
          break
  
print("buffer initialized")


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 88, 80, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 22, 20, 32)   2080        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 11, 10, 64)   32832       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 11, 10, 64)   36928       conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:

for e in tqdm(range(0, episodes)):
    total_reward = 0
    game_score = 0
    state = preprocess(env.reset())
    frame_stack = deque(maxlen=4)
    frame_stack.append(state)
    epochs = 0
    
    for skip in range(90):
        env.step(0)
    
    for time in range(20000):
        total_time += 1
        epochs += 1
        
        if total_time % agent.update_rate == 0:
            agent.update_target_model()
        
        state = sum(frame_stack)/len(frame_stack)
        
        action = agent.act(np.expand_dims(state.reshape(88, 80, 1), axis=0))
        next_state, reward, Terminal, _ = env.step(action)
        
        next_state = preprocess(next_state)
        frame_stack.append(next_state)
        next_state = sum(frame_stack)/len(frame_stack)
        
        td_error = agent.calculate_td_error(state, action, reward, next_state, Terminal)

        agent.store(state, action, reward, next_state, Terminal, td_error)
        
        state = next_state
        total_reward += reward
        
        if Terminal:
            all_rewards += total_reward
            if wb:
              wandb.log({'episodes:': e, 'episodic_reward': total_reward})
            print("episode: {}/{}, reward: {}".format(e+1, episodes, total_reward))
            break

        if epochs % 5 == 0:    
          agent.replay(batch_size)

    if (e+1) % 500 == 0:
      print("model saved on episode", e)
      agent.save("")

  0%|          | 1/20000 [02:09<720:02:13, 129.61s/it]

episode: 1/20000, reward: 310.0


  0%|          | 2/20000 [04:39<754:28:40, 135.82s/it]

episode: 2/20000, reward: 240.0


  0%|          | 3/20000 [07:22<799:38:39, 143.96s/it]

...

episode: 326/20000, reward: 250.0


  2%|▏         | 327/20000 [11:53:47<885:20:06, 162.01s/it]

episode: 327/20000, reward: 1980.0


  2%|▏         | 328/20000 [11:57:19<967:27:42, 177.05s/it]

episode: 328/20000, reward: 610.0
